<a href="https://colab.research.google.com/github/AleAguirreM/Proyecto-ACCIDENTALIDAD-VIAL/blob/main/Proyecto_Movilidad_Vial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>